In [18]:
import pandas as pd
import numpy as np
import re
import ast

df_reviews = pd.read_csv('../../data/raw/reviews.csv')
df_reviews.head(10)

,uid,profile,anime_uid,text,score,scores,link
0,255938,DesolatePsyche,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=255938
1,259117,baekbeans,34599,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=259117
2,253664,skrn,28891,\n \n \n \n ...,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=253664
3,8254,edgewalker00,2904,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '9...",https://myanimelist.net/reviews.php?id=8254
4,291149,aManOfCulture99,4181,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=291149
5,10046,eneri,2904,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '9', 'Animation': '...",https://myanimelist.net/reviews.php?id=10046
6,247454,Waffle_Empress,16664,\n \n \n \n ...,6,"{'Overall': '6', 'Story': '6', 'Animation': '9...",https://myanimelist.net/reviews.php?id=247454
7,140903,NIGGER_BONER,2904,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=140903
8,23791,jchang,2904,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=23791
9,25115,shadowsplat,4181,\n \n \n \n ...,4,"{'Overall': '4', 'Story': '8', 'Animation': '9...",https://myanimelist.net/reviews.php?id=25115


In [19]:
df_reviews.iloc[2, 3]

"\n           \n         \n           \n             \n           \n         \n         \n           more pics \n         \n       \n         \n       \n         \n           Overall \n           7 \n         \n         \n           Story \n           7 \n         \n                   \n             Animation \n             9 \n           \n           \n             Sound \n             8 \n           \n                 \n           Character \n           8 \n         \n         \n           Enjoyment \n           8 \n         \n       \n     \n\n                    \n    Art 9/10: It is great, especially the actions during the critical moments in the game. However, it doesn't WOW me like the best in class. \r\n \r\nSound 8/10: Very good voice acting, good sound effective during actions, good music that help bring out emotions and some of the op and ed songs are great. However, there is no sound track that are memorable. Can't give it a higher score. \r\n \r\nCharacter 8/10: There are 

In [20]:
df_reviews = df_reviews.drop_duplicates()
df_reviews.isnull().sum()

uid          0
profile      0
anime_uid    0
text         0
score        0
scores       0
link         0
dtype: int64

#### WOW, 0 Missing values

In [21]:
COMMON_BOILERPLATE = {
    "read more",
    "read less",
    "this review contains spoilers",
    "show less",
    "show more",
}

RATING_LABELS = r'(?:Overall|Story|Animation|Sound|Character|Enjoyment)'

def _clean_review_text(s: object) -> str:
    # Guard: handle NaN
    if pd.isna(s):
        return ''
    text = str(s)
    if text.strip() == '':
        return ''

    # Remove HTML-like tags and common entities
    text = re.sub(r'<[^>]*>', ' ', text)
    text = re.sub(r'&nbsp;|&amp;|&lt;|&gt;|&#39;|&quot;', ' ', text)

    # Normalize line endings and collapse multiple newlines temporarily
    text = text.replace('\r', '\n')
    text = re.sub(r'\n+', '\n', text)

    # Remove leading rating boilerplate like:
    text = re.sub(r'^\s*(?:more pics\s*)?(?:' + RATING_LABELS + r'\s*\d+\s*){1,}\s*', '', text, flags=re.IGNORECASE)
    # Also remove any standalone leading 'more pics' left behind
    text = re.sub(r'^\s*more pics\s*', '', text, flags=re.IGNORECASE)

    # Remove trailing 'Helpful' or similar single-word boilerplate at the end
    #    e.g. 'Helpful', 'helpful', or 'Helpful 123' etc.
    text = re.sub(r'(?:\bHelpful\b(?:\s*\d+)?)\s*$','', text, flags=re.IGNORECASE)

    # Split into lines, trim, drop known short boilerplate lines from start/end
    lines = [ln.strip() for ln in text.split('\n') if ln.strip() != '']
    # remove known COMMON_BOILERPLATE lines at start
    while len(lines) > 0 and lines[0].lower() in COMMON_BOILERPLATE:
        lines.pop(0)
    # remove known COMMON_BOILERPLATE lines at end
    while len(lines) > 0 and lines[-1].lower() in COMMON_BOILERPLATE:
        lines.pop()

    # Join remaining lines with a single space
    text = ' '.join(lines)

    # Replace any leftover sequences of whitespace (including newlines) with a single space
    text = re.sub(r"\s+", ' ', text)

    # Final strip
    return text.strip()

if 'text' in df_reviews.columns:
    # prefer cleaning original 'text' column; if 'text_clean' already exists overwrite it
    df_reviews['text_clean'] = df_reviews['text'].apply(_clean_review_text)
elif 'text_clean' in df_reviews.columns:
    # fallback: clean existing text_clean if that was present
    df_reviews['text_clean'] = df_reviews['text_clean'].apply(_clean_review_text)
else:
    df_reviews['text_clean'] = ''

print('Cleaned text into text_clean. Sample:')
df_reviews['text_clean'].head(10)


Cleaned text into text_clean. Sample:


0    First things first. My "reviews" system is exp...
1    Let me start off by saying that Made in Abyss ...
2    Art 9/10: It is great, especially the actions ...
3    Story taking place 1 yr from where season 1 tr...
4    Kyoto Animations greatest strength is being ab...
5    When I first saw Code Geass I was like omgzxwt...
6    As someone who loves Studio Ghibli and its mov...
7    Code Geass R2 is sitting pretty on myanimelist...
8    code geass is one of those series that everybo...
9    Okay, so I was pleased to see that Clannad act...
Name: text_clean, dtype: object

In [22]:
df_reviews = df_reviews.drop(columns=['link', 'text'])
df_reviews.head(10)

,uid,profile,anime_uid,score,scores,text_clean
0,255938,DesolatePsyche,34096,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...","First things first. My ""reviews"" system is exp..."
1,259117,baekbeans,34599,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",Let me start off by saying that Made in Abyss ...
2,253664,skrn,28891,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...","Art 9/10: It is great, especially the actions ..."
3,8254,edgewalker00,2904,9,"{'Overall': '9', 'Story': '9', 'Animation': '9...",Story taking place 1 yr from where season 1 tr...
4,291149,aManOfCulture99,4181,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",Kyoto Animations greatest strength is being ab...
5,10046,eneri,2904,10,"{'Overall': '10', 'Story': '9', 'Animation': '...",When I first saw Code Geass I was like omgzxwt...
6,247454,Waffle_Empress,16664,6,"{'Overall': '6', 'Story': '6', 'Animation': '9...",As someone who loves Studio Ghibli and its mov...
7,140903,NIGGER_BONER,2904,8,"{'Overall': '8', 'Story': '7', 'Animation': '9...",Code Geass R2 is sitting pretty on myanimelist...
8,23791,jchang,2904,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",code geass is one of those series that everybo...
9,25115,shadowsplat,4181,4,"{'Overall': '4', 'Story': '8', 'Animation': '9...","Okay, so I was pleased to see that Clannad act..."


In [23]:
type(df_reviews.iloc[0, 4])

str

In [25]:
SCORE_KEYS = ['Overall', 'Story', 'Animation', 'Sound', 'Character', 'Enjoyment']

def _parse_scores_dict(s: object) -> dict:
    if pd.isna(s):
        return {}
    # Accept dicts already parsed
    if isinstance(s, dict):
        return s
    try:
        # Safely evaluate Python literal (e.g. "{'Overall': '8', ...}")
        parsed = ast.literal_eval(s)
        if isinstance(parsed, dict):
            return parsed
        else:
            return {}
    except Exception:
        return {}

if 'scores' in df_reviews.columns:
    parsed_scores = df_reviews['scores'].apply(_parse_scores_dict)
    for k in SCORE_KEYS:
        col_name = f"{k.lower()}_score"
        df_reviews[col_name] = parsed_scores.apply(lambda d: d.get(k))
        # convert to numeric, coerce malformed to NA, use pandas nullable integer dtype
        df_reviews[col_name] = pd.to_numeric(df_reviews[col_name], errors='coerce').astype('Int64')
else:
    # create empty columns if scores missing
    for k in SCORE_KEYS:
        col_name = f"{k.lower()}_score"
        df_reviews[col_name] = pd.Series([pd.NA] * len(df_reviews), dtype='Int64')

print('Added score columns:', [f"{k.lower()}_score" for k in SCORE_KEYS])
df_reviews[[f"{k.lower()}_score" for k in SCORE_KEYS]].head()


Added score columns: ['overall_score', 'story_score', 'animation_score', 'sound_score', 'character_score', 'enjoyment_score']


,overall_score,story_score,animation_score,sound_score,character_score,enjoyment_score
0,8,8,8,10,9,8
1,10,10,10,10,10,10
2,7,7,9,8,8,8
3,9,9,9,10,10,9
4,10,10,8,9,10,10


In [27]:
df_reviews = df_reviews.drop(columns=['score'])
df_reviews.head(10)

,uid,profile,anime_uid,scores,text_clean,overall_score,story_score,animation_score,sound_score,character_score,enjoyment_score
0,255938,DesolatePsyche,34096,"{'Overall': '8', 'Story': '8', 'Animation': '8...","First things first. My ""reviews"" system is exp...",8,8,8,10,9,8
1,259117,baekbeans,34599,"{'Overall': '10', 'Story': '10', 'Animation': ...",Let me start off by saying that Made in Abyss ...,10,10,10,10,10,10
2,253664,skrn,28891,"{'Overall': '7', 'Story': '7', 'Animation': '9...","Art 9/10: It is great, especially the actions ...",7,7,9,8,8,8
3,8254,edgewalker00,2904,"{'Overall': '9', 'Story': '9', 'Animation': '9...",Story taking place 1 yr from where season 1 tr...,9,9,9,10,10,9
4,291149,aManOfCulture99,4181,"{'Overall': '10', 'Story': '10', 'Animation': ...",Kyoto Animations greatest strength is being ab...,10,10,8,9,10,10
5,10046,eneri,2904,"{'Overall': '10', 'Story': '9', 'Animation': '...",When I first saw Code Geass I was like omgzxwt...,10,9,10,10,10,10
6,247454,Waffle_Empress,16664,"{'Overall': '6', 'Story': '6', 'Animation': '9...",As someone who loves Studio Ghibli and its mov...,6,6,9,8,6,6
7,140903,NIGGER_BONER,2904,"{'Overall': '8', 'Story': '7', 'Animation': '9...",Code Geass R2 is sitting pretty on myanimelist...,8,7,9,0,0,9
8,23791,jchang,2904,"{'Overall': '10', 'Story': '10', 'Animation': ...",code geass is one of those series that everybo...,10,10,9,10,10,10
9,25115,shadowsplat,4181,"{'Overall': '4', 'Story': '8', 'Animation': '9...","Okay, so I was pleased to see that Clannad act...",4,8,9,8,5,4


In [28]:
df_reviews.columns

Index(['uid', 'profile', 'anime_uid', 'scores', 'text_clean', 'overall_score',
       'story_score', 'animation_score', 'sound_score', 'character_score',
       'enjoyment_score'],
      dtype='object')

In [32]:
# want to check cols for animes ready dataset
df_animes = pd.read_parquet("../../data/processed/animes_processed.parquet")
print(df_animes.columns[:80])

Index(['uid', 'title', 'synopsis', 'aired', 'episodes', 'members',
       'popularity', 'score', 'genres_list', 'genre_action', 'genre_adventure',
       'genre_cars', 'genre_comedy', 'genre_dementia', 'genre_demons',
       'genre_drama', 'genre_ecchi', 'genre_fantasy', 'genre_game',
       'genre_harem', 'genre_hentai', 'genre_historical', 'genre_horror',
       'genre_josei', 'genre_kids', 'genre_magic', 'genre_martial_arts',
       'genre_mecha', 'genre_military', 'genre_music', 'genre_mystery',
       'genre_parody', 'genre_police', 'genre_psychological', 'genre_romance',
       'genre_samurai', 'genre_school', 'genre_sci_fi', 'genre_seinen',
       'genre_shoujo', 'genre_shoujo_ai', 'genre_shounen', 'genre_shounen_ai',
       'genre_slice_of_life', 'genre_space', 'genre_sports',
       'genre_super_power', 'genre_supernatural', 'genre_thriller',
       'genre_vampire', 'genre_yaoi', 'genre_yuri', 'source', 'n_genres',
       'rare_genre_flag', 'genre_entropy', 'synopsis_clean', '

In [33]:
df_reviews.to_parquet("../../data/processed/reviews_processed.parquet")